In [10]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
fitbit_df = pd.read_csv('train.csv')
fitbit_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


In [11]:
# Generate our categorical variable list
fitbit_cat = fitbit_df.dtypes[fitbit_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
fitbit_df[fitbit_cat].nunique()

Activity    6
dtype: int64

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(fitbit_df[fitbit_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(fitbit_cat)
encode_df

,Activity_LAYING,Activity_SITTING,Activity_STANDING,Activity_WALKING,Activity_WALKING_DOWNSTAIRS,Activity_WALKING_UPSTAIRS
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
7347,0.0,0.0,0.0,0.0,0.0,1.0
7348,0.0,0.0,0.0,0.0,0.0,1.0
7349,0.0,0.0,0.0,0.0,0.0,1.0
7350,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# Merge one-hot encoded features and drop the originals
fitbit_df = fitbit_df.merge(encode_df,left_index=True, right_index=True)
fitbit_df = fitbit_df.drop(fitbit_cat,1)
fitbit_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity_LAYING,Activity_SITTING,Activity_STANDING,Activity_WALKING,Activity_WALKING_DOWNSTAIRS,Activity_WALKING_UPSTAIRS
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.841247,0.179941,-0.058627,1,0.0,0.0,1.0,0.0,0.0,0.0
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.844788,0.180289,-0.054317,1,0.0,0.0,1.0,0.0,0.0,0.0
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.848933,0.180637,-0.049118,1,0.0,0.0,1.0,0.0,0.0,0.0
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.848649,0.181935,-0.047663,1,0.0,0.0,1.0,0.0,0.0,0.0
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.847865,0.185151,-0.043892,1,0.0,0.0,1.0,0.0,0.0,0.0


In [24]:
# Predicting laying
y = fitbit_df.Activity_LAYING.values
X = fitbit_df.values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Create the SVM model
svm = SVC(kernel='linear')

In [26]:
# Train the model
svm.fit(X_train, y_train)

SVC(kernel='linear')

In [27]:
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.986


In [28]:
# Build and compile deep learning model
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
173/173 [==============================] - 1s 1ms/step - loss: 0.4084 - accuracy: 0.8330
Epoch 2/50
173/173 [==============================] - 0s 1ms/step - loss: 0.0231 - accuracy: 0.9989
Epoch 3/50
173/173 [==============================] - 0s 1ms/step - loss: 0.0063 - accuracy: 0.9996
Epoch 4/50
173/173 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 0.9998
Epoch 5/50
173/173 [==============================] - 0s 1ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 6/50
173/173 [==============================] - 0s 1ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 7/50
173/173 [==============================] - 0s 1ms/step - loss: 7.0513e-04 - accuracy: 1.0000
Epoch 8/50
173/173 [==============================] - 0s 1ms/step - loss: 5.0723e-04 - accuracy: 1.0000
Epoch 9/50
173/173 [==============================] - 0s 1ms/step - loss: 4.0389e-04 - accuracy: 1.0000
Epoch 10/50
173/173 [==============================] - 0s 1ms/step - loss: 3.0629e-04 - 

### This is only analyzing one categorical feature of the data set right now, my first goal will incorporate all of the categorical features intended for the model.

Which model did you choose and why?
    SVM it's efficient and could potentially give a accurate result
    
How are you training your model?
    Training the model on 70% of the total dataset, and use remaining 30% for testing.
    
What is the model's accuracy?
    Currently 100%, but as said before, only one categorical feature of the data set is being used right now and that will need to be changed.